In [1]:
!pip install kaggle

In [2]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

!apt-get update -qq
!apt-get install fonts-nanum* -qq

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import warnings
warnings.filterwarnings(action='ignore')

path = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf' # 나눔 고딕
font_name = fm.FontProperties(fname=path, size=10).get_name() # 기본 폰트 사이즈 : 10
plt.rc('font', family=font_name)

fm.fontManager.addfont(path)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (4,577 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126675 files and dire

In [4]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

os.environ['KAGGLE_USERNAME'] = 'starsin'
os.environ['KAGGLE_KEY'] = 'ff559a02afd29d67f19bc4001d5a69f7'

api = KaggleApi()
api.authenticate()


# 2. 대회 데이터 다운로드
competition_name = "ai05-level1-project"
api.competition_download_files(competition_name, path="./data")

In [5]:
import zipfile

zip_path = "./data/ai05-level1-project.zip"
extract_dir = "./data/ai05-level1-project"

os.makedirs(extract_dir, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_dir)

In [3]:
# === 0) 공통 임포트 & 경로 ===
import os, json, random, time
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torchvision.ops as ops

# 경로
base_path = "./data/ai05-level1-project"
img_dir   = os.path.join(base_path, "train_images")
test_dir  = os.path.join(base_path, "test_images")
train_json = "./merged_coco/train_all.json"   # COCO 합본(없으면 맨 아래 옵션 셀 참고)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cuda


In [4]:
# === 옵션) train_annotations 전체를 하나의 COCO로 병합 ===
train_ann_dir = os.path.join(base_path, "train_annotations")
merged = {"images": [], "annotations": [], "categories": []}
cat_map = {}
imgid_remap = {}   # (file_name, orig_image_id) -> new_image_id
ann_id = 1
new_img_id = 1

for folder in os.listdir(train_ann_dir):
    if not folder.endswith("_json"):
        continue
    ann_dir = os.path.join(train_ann_dir, folder)
    for dp, _, files in os.walk(ann_dir):
        for f in files:
            if not f.endswith(".json"):
                continue
            with open(os.path.join(dp, f), "r") as fh:
                j = json.load(fh)
            images = j.get("images", [])
            anns   = j.get("annotations", [])
            cats   = j.get("categories", [])
            for c in cats:
                if c["id"] not in cat_map:
                    cat_map[c["id"]] = c
            for im in images:
                file_name = os.path.basename(im.get("file_name") or im.get("imgfile"))
                key = (file_name, im["id"])
                if key not in imgid_remap:
                    imgid_remap[key] = new_img_id
                    merged["images"].append({
                        "id": new_img_id,
                        "file_name": file_name,
                        "width": im.get("width", 1024),
                        "height": im.get("height", 1024)
                    })
                    new_img_id += 1
            for a in anns:
                for im in images:
                    if im["id"] == a["image_id"]:
                        file_name = os.path.basename(im.get("file_name") or im.get("imgfile"))
                        mapped = imgid_remap[(file_name, im["id"])]
                        new_a = dict(a)
                        new_a["id"] = ann_id; ann_id += 1
                        new_a["image_id"] = mapped
                        merged["annotations"].append(new_a)
                        break

merged["categories"] = list(cat_map.values())
os.makedirs("./merged_coco", exist_ok=True)
with open("./merged_coco/train_all.json", "w") as f:
    json.dump(merged, f, indent=2)
print("✅ COCO 병합 완료:", len(merged["images"]), "images,", len(merged["annotations"]), "anns,", len(merged["categories"]), "cats")

✅ COCO 병합 완료: 1489 images, 4526 anns, 73 cats


In [5]:
# === 1) Dataset / Loader (bbox -> mask 자동 생성) ===
def get_transform(train=True):
    # 박스를 건드리지 않는 photometric 증강만 사용 (안전)
    aug = [T.ToTensor()]
    if train:
        aug += [
            T.ColorJitter(brightness=0.15, contrast=0.15, saturation=0.15, hue=0.03),
            T.RandomAdjustSharpness(sharpness_factor=1.5, p=0.30),
            T.GaussianBlur(kernel_size=3, sigma=(0.1, 1.5)),
            T.RandomAutocontrast(p=0.25),
        ]
    return T.Compose(aug)

def boxes_to_rect_masks(size_hw, boxes_xyxy):
    """
    size_hw: (H, W)
    boxes_xyxy: float/np.array [N,4] (x1,y1,x2,y2)
    return: uint8 mask [N, H, W] (0/1)
    """
    H, W = size_hw
    masks = []
    for (x1, y1, x2, y2) in boxes_xyxy:
        x1, y1, x2, y2 = map(float, (x1, y1, x2, y2))
        x1 = max(0, min(W-1, x1)); x2 = max(0, min(W-1, x2))
        y1 = max(0, min(H-1, y1)); y2 = max(0, min(H-1, y2))
        if x2 <= x1 or y2 <= y1:
            masks.append(np.zeros((H, W), dtype=np.uint8))
            continue
        m = np.zeros((H, W), dtype=np.uint8)
        m[int(y1):int(y2), int(x1):int(x2)] = 1
        masks.append(m)
    if len(masks)==0:
        return np.zeros((0, H, W), dtype=np.uint8)
    return np.stack(masks, axis=0)

class CocoLikeMaskDataset(torch.utils.data.Dataset):
    """
    Mask R-CNN용 Dataset.
    - COCO 'categories'의 id를 0-based index로 매핑하여 labels 생성
    - segmentation이 없어도 bbox를 사각형 마스크로 rasterize
    """
    def __init__(self, img_dir, coco_json, transforms=None, train=True):
        with open(coco_json, "r") as f:
            coco = json.load(f)

        self.img_dir = img_dir
        self.transforms = transforms
        self.train = train

        self.images  = {im["id"]: im for im in coco["images"]}
        self.img_ids = list(self.images.keys())

        self.anns_by_img = {im_id: [] for im_id in self.img_ids}
        for a in coco["annotations"]:
            self.anns_by_img[a["image_id"]].append(a)

        self.cat_ids = sorted({c["id"] for c in coco["categories"]})
        self.cat_id_to_idx = {cid: i for i, cid in enumerate(self.cat_ids)}   # 0..K-1
        self.idx_to_catid  = {i: cid for cid, i in self.cat_id_to_idx.items()}

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id  = self.img_ids[idx]
        im_meta = self.images[img_id]
        file_name = im_meta["file_name"]
        W = im_meta.get("width", 1024)
        H = im_meta.get("height", 1024)

        img_path = os.path.join(self.img_dir, file_name)
        img = Image.open(img_path).convert("RGB")

        boxes, labels, areas, iscrowd = [], [], [], []
        for a in self.anns_by_img.get(img_id, []):
            if "bbox" not in a or not a["bbox"]:
                continue
            x, y, w, h = a["bbox"]
            boxes.append([x, y, x+w, y+h])
            labels.append(self.cat_id_to_idx.get(a["category_id"], 0))
            areas.append(a.get("area", w*h))
            iscrowd.append(a.get("iscrowd", 0))

        boxes_np = np.array(boxes, dtype=np.float32) if boxes else np.zeros((0,4), np.float32)
        masks_np = boxes_to_rect_masks((H, W), boxes_np)  # ✅ bbox→mask

        target = {
            "boxes":    torch.as_tensor(boxes_np, dtype=torch.float32),
            "labels":   torch.as_tensor(labels, dtype=torch.int64) if labels else torch.zeros((0,), dtype=torch.int64),
            "masks":    torch.as_tensor(masks_np, dtype=torch.uint8),    # [N,H,W], 0/1
            "image_id": torch.tensor([img_id]),
            "area":     torch.as_tensor(areas, dtype=torch.float32) if areas else torch.zeros((0,), dtype=torch.float32),
            "iscrowd":  torch.as_tensor(iscrowd, dtype=torch.int64) if iscrowd else torch.zeros((0,), dtype=torch.int64),
        }

        if self.transforms:
            img = self.transforms(img)  # photometric only
        else:
            img = T.ToTensor()(img)

        return img, target

def collate_fn(batch):
    return tuple(zip(*batch))

# 로더
full_ds = CocoLikeMaskDataset(img_dir, train_json, transforms=get_transform(train=True), train=True)
indices = list(range(len(full_ds)))
random.Random(0).shuffle(indices)
split = int(0.9*len(indices))
train_idx, val_idx = indices[:split], indices[split:]

train_ds = torch.utils.data.Subset(full_ds, train_idx)
val_ds   = torch.utils.data.Subset(CocoLikeMaskDataset(img_dir, train_json, transforms=get_transform(train=False), train=False),
                                   val_idx)

train_loader = torch.utils.data.DataLoader(train_ds, batch_size=8, shuffle=True,  num_workers=2, collate_fn=collate_fn)
val_loader   = torch.utils.data.DataLoader(val_ds,   batch_size=8, shuffle=False, num_workers=2, collate_fn=collate_fn)

num_classes = len(full_ds.cat_ids)  # ✅ full_ds는 Dataset 객체
print("✅ train:", len(train_ds), "val:", len(val_ds), "classes:", num_classes)

✅ train: 1340 val: 149 classes: 73


In [6]:
# === 2) Mask R-CNN 초기화 ===
model = torchvision.models.detection.maskrcnn_resnet50_fpn_v2(
    weights=None,                # 필요하면 weights="DEFAULT"
    num_classes=num_classes      # torchvision: 배경은 내부 처리
).to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.AdamW(params, lr=2.5e-4, weight_decay=1e-4)
lr_sched  = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=16)

print("Model ready.")

Model ready.


In [7]:
# === 3) EarlyStopping & Train Loop ===
class EarlyStopping:
    def __init__(self, patience=3, delta=0.0):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best = None
        self.early_stop = False

    def __call__(self, val_loss):
        if (self.best is None) or ((self.best - val_loss) > self.delta):
            self.best = val_loss
            self.counter = 0
        else:
            self.counter += 1
            print(f"⚠️ EarlyStopping {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True

early_stopper = EarlyStopping(patience=3, delta=1e-4)

EPOCHS = 20
for epoch in range(1, EPOCHS+1):
    # ---- train ----
    model.train()
    total_loss = 0.0
    for imgs, targets in train_loader:
        imgs = [im.to(device) for im in imgs]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # label 범위 방어
        for t in targets:
            if t["labels"].numel() > 0:
                assert int(t["labels"].min()) >= 0
                assert int(t["labels"].max()) < num_classes

        loss_dict = model(imgs, targets)   # dict: classification, box_reg, mask, ...
        loss = sum(loss_dict.values())

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        total_loss += float(loss.item())

    lr_sched.step()
    train_loss = total_loss / max(1, len(train_loader))

    # ---- validation loss (손실 계산 위해 train 유지 + no_grad) ----
    model.train()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs = [im.to(device) for im in imgs]
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = model(imgs, targets)
            loss = sum(loss_dict.values())
            val_loss += float(loss.item())
    val_loss /= max(1, len(val_loader))

    print(f"[{epoch}/{EPOCHS}] train_loss={train_loss:.4f} | val_loss={val_loss:.4f}")

    early_stopper(val_loss)
    if early_stopper.early_stop:
        print(f"⏹ Early stopping at epoch {epoch}")
        break

# 저장
os.makedirs("./checkpoints", exist_ok=True)
ckpt_path = "./checkpoints/maskrcnn_pills.pth"
torch.save(model.state_dict(), ckpt_path)
print("💾 saved:", ckpt_path)

[1/20] train_loss=1.3235 | val_loss=0.6550
[2/20] train_loss=0.6427 | val_loss=0.3274
[3/20] train_loss=0.4603 | val_loss=0.2811
[4/20] train_loss=0.3776 | val_loss=0.2613
[5/20] train_loss=0.3550 | val_loss=0.2005
[6/20] train_loss=0.3313 | val_loss=0.1826
[7/20] train_loss=0.3183 | val_loss=0.1548
[8/20] train_loss=0.2980 | val_loss=0.1607
⚠️ EarlyStopping 1/3
[9/20] train_loss=0.2813 | val_loss=0.1358
[10/20] train_loss=0.2704 | val_loss=0.1166
[11/20] train_loss=0.2535 | val_loss=0.1166
⚠️ EarlyStopping 1/3
[12/20] train_loss=0.2386 | val_loss=0.1048
[13/20] train_loss=0.2309 | val_loss=0.1110
⚠️ EarlyStopping 1/3
[14/20] train_loss=0.2307 | val_loss=0.1038
[15/20] train_loss=0.2304 | val_loss=0.0998
[16/20] train_loss=0.2320 | val_loss=0.1002
⚠️ EarlyStopping 1/3
[17/20] train_loss=0.2273 | val_loss=0.0999
⚠️ EarlyStopping 2/3
[18/20] train_loss=0.2235 | val_loss=0.0995
[19/20] train_loss=0.2263 | val_loss=0.1009
⚠️ EarlyStopping 1/3
[20/20] train_loss=0.2304 | val_loss=0.1041
⚠️ 

In [9]:
# === 4) 추론 유틸 (약당 1개만 남기기) ===

# 0-based idx ↔ category_id 매핑
idx_to_catid = getattr(full_ds, "idx_to_catid", None)
if idx_to_catid is None:
    with open(train_json, "r") as f:
        coco_tmp = json.load(f)
    cat_ids_list = sorted({c["id"] for c in coco_tmp["categories"]})
    idx_to_catid = {i: cid for i, cid in enumerate(cat_ids_list)}

def keep_one_per_category(B, L_cat, S, how="area"):
    if len(B)==0:
        return B, L_cat, S
    B = np.asarray(B, np.float32)
    L_cat = np.asarray(L_cat, np.int64)
    S = np.asarray(S, np.float32)
    kept_b, kept_l, kept_s = [], [], []
    for cat in np.unique(L_cat):
        m = (L_cat == cat)
        bb = B[m]; ss = S[m]
        if bb.shape[0]==1:
            idx = 0
        else:
            areas = (bb[:,2]-bb[:,0]) * (bb[:,3]-bb[:,1])
            if how == "score":
                idx = int(np.argmax(ss))
            elif how == "area_score":
                score = (areas**0.7) * (np.clip(ss,1e-6,None)**0.3)
                idx = int(np.argmax(score))
            else:
                idx = int(np.argmax(areas))
        kept_b.append(bb[idx]); kept_l.append(cat); kept_s.append(ss[idx])
    return (np.vstack(kept_b).astype(np.float32),
            np.array(kept_l, np.int64),
            np.array(kept_s, np.float32))

@torch.no_grad()
def infer_maskrcnn_one(model, pil_img, score_thr=0.30):
    x = T.ToTensor()(pil_img).to(device)
    model.eval()
    out = model([x])[0]
    b  = out["boxes"].detach().cpu().numpy()
    l  = out["labels"].detach().cpu().numpy()  # 0-based idx
    s  = out["scores"].detach().cpu().numpy()
    m  = out.get("masks", None)
    if m is not None:
        m = m.detach().cpu().numpy()  # [N,1,H,W] (sigmoid)
    # 필터
    keep = (s >= score_thr) & (l != 0)  # 0==배경 방지용
    return b[keep], l[keep], s[keep], (m[keep] if m is not None else None)

In [10]:
# === 5) Test 추론 & CSV (약당 1개만, 시각화) ===
score_thr = 0.30
csv_out = "./stage2_maskrcnn_predictions.csv"

rows = []
annotation_id = 1

file_list = sorted([fn for fn in os.listdir(test_dir)
                    if fn.lower().endswith((".png",".jpg",".jpeg"))])

VIS_N, shown = 8, 0

for fn in file_list:
    img_path = os.path.join(test_dir, fn)
    image_id = os.path.splitext(fn)[0]
    img = Image.open(img_path).convert("RGB")

    B, L_idx, S, _ = infer_maskrcnn_one(model, img, score_thr=score_thr)
    if len(B)==0:
        continue

    # 모델 라벨(0-based idx) -> 실제 카테고리 id
    L_cat = np.array([idx_to_catid[int(li)] for li in L_idx], dtype=np.int64)

    # 동일 약(category_id)당 1개만 남기기 (면적 기준)
    B, L_cat, S = keep_one_per_category(B, L_cat, S, how="area")

    # CSV 적재
    for (x1,y1,x2,y2), lab, sc in zip(B, L_cat, S):
        rows.append({
            "annotation_id": annotation_id,
            "image_id": image_id,           # 확장자 제거
            "category_id": int(lab),
            "bbox_x": round(float(x1), 2),
            "bbox_y": round(float(y1), 2),
            "bbox_w": round(float(x2-x1), 2),
            "bbox_h": round(float(y2-y1), 2),
            "score":  round(float(sc), 3)
        })
        annotation_id += 1

    # 시각화
    if shown < VIS_N:
        fig, ax = plt.subplots(1,1, figsize=(7,7))
        ax.imshow(img); ax.axis("off")
        for (x1,y1,x2,y2), lab, sc in zip(B, L_cat, S):
            rect = patches.Rectangle((x1,y1), x2-x1, y2-y1, linewidth=2, edgecolor='lime', facecolor='none')
            ax.add_patch(rect)
            ax.text(x1, max(0,y1-5), f"{int(lab)} ({sc:.2f})",
                    fontsize=9, color="white",
                    bbox=dict(facecolor="black", alpha=0.6, pad=1))
        ax.set_title(f"{fn} — Mask R-CNN per-drug ONE", fontsize=11)
        plt.show()
        shown += 1

df = pd.DataFrame(rows, columns=[
    "annotation_id","image_id","category_id","bbox_x","bbox_y","bbox_w","bbox_h","score"
])
df.to_csv(csv_out, index=False, encoding="utf-8-sig")
print(f"✅ CSV 저장: {csv_out} (rows={len(df)})")

Output hidden; open in https://colab.research.google.com to view.